In [4]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
login()

# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to("cpu") # or "cpu"



/Users/alprielse/src/sandboxed/2024-08-20_esm3-toying/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Fetching 22 files: 100%|██████████| 22/22 [00:00<00:00, 19418.07it/s]


In [6]:

import biotite.sequence as seq
import biotite.sequence.align as align
import biotite.sequence.graphics as graphics
from getpass import getpass
import matplotlib.pyplot as pl
import py3Dmol
import torch

from esm.sdk import client
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig
from esm.utils.structure.protein_chain import ProteinChain

In [7]:
prompt = "___________________________________________________DQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPP___________________________________________________________"
protein = ESMProtein(sequence=prompt)
# Generate the sequence, then the structure. This will iteratively unmask the sequence track.
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8, temperature=0.7))
#

100%|██████████| 8/8 [00:31<00:00,  3.91s/it]


Prompt is based off of this thing
https://www.rcsb.org/structure/1qy3


In [8]:
template_gfp = ESMProtein.from_protein_chain(
  ProteinChain.from_rcsb("1qy3", chain_id="A")
)
template_gfp_tokens = model.encode(template_gfp)

In [11]:
template_gfp

ESMProtein(sequence='KGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQEATISFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGI', secondary_structure=None, sasa=None, function_annotations=None, coordinates=tensor([[[  1.0950,  24.0170,  16.2260],
         [  0.2150,  22.8420,  15.9660],
         [  0.4970,  22.2670,  14.5770],
         ...,
         [     nan,      nan,      nan],
         [ -2.1380,  19.1750,  15.5330],
         [     nan,      nan,      nan]],

        [[  0.6520,  20.9440,  14.5320],
         [  0.9550,  20.2360,  13.3010],
         [ -0.0280,  20.2910,  12.1430],
         ...,
         [     nan,      nan,      nan],
         [     nan,      nan,      nan],
         [     nan,      nan,      nan]],

        [[ -1.3270,  20.4360,  12.4260],
         [ -2.3520,  20.4950,  11.3720],
         [ -2.2650,  21.7120,  10.4440],
         ...,
         [  

In [13]:
protein = model.generate(template_gfp, GenerationConfig(track="structure", num_steps=8))

100%|██████████| 8/8 [00:39<00:00,  4.97s/it]


In [14]:
protein.to_pdb("./1qy3.pdb")

In [15]:
view = py3Dmol.view(width=600, height=600)
view.addModel(protein.to_pdb_string(), "pdb")
view.setStyle({"cartoon": {"color": "lightgreen"}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [16]:
print("Sequence tokens:")
print("    ", ", ".join([
    str(token) for token in template_gfp_tokens.sequence.tolist()
]))

Sequence tokens:
     0, 15, 6, 9, 9, 4, 18, 11, 6, 7, 7, 14, 12, 4, 7, 9, 4, 13, 6, 13, 7, 17, 6, 21, 15, 18, 8, 7, 8, 6, 9, 6, 9, 6, 13, 5, 11, 19, 6, 15, 4, 11, 4, 15, 18, 12, 23, 11, 11, 6, 15, 4, 14, 7, 14, 22, 14, 11, 4, 7, 11, 11, 4, 11, 19, 6, 7, 16, 23, 18, 8, 10, 19, 14, 13, 21, 20, 15, 16, 21, 13, 18, 18, 15, 8, 5, 20, 14, 9, 6, 19, 7, 16, 9, 5, 11, 12, 8, 18, 15, 13, 13, 6, 17, 19, 15, 11, 10, 5, 9, 7, 15, 18, 9, 6, 13, 11, 4, 7, 17, 10, 12, 9, 4, 15, 6, 12, 13, 18, 15, 9, 13, 6, 17, 12, 4, 6, 21, 15, 4, 9, 19, 17, 19, 17, 8, 21, 17, 7, 19, 12, 11, 5, 13, 15, 16, 15, 17, 6, 12, 15, 5, 17, 18, 15, 12, 10, 21, 17, 12, 9, 13, 6, 8, 7, 16, 4, 5, 13, 21, 19, 16, 16, 17, 11, 14, 12, 6, 13, 6, 14, 7, 4, 4, 14, 13, 17, 21, 19, 4, 8, 11, 16, 8, 5, 4, 8, 15, 13, 14, 17, 9, 15, 10, 13, 21, 20, 7, 4, 4, 9, 18, 7, 11, 5, 5, 6, 12, 2


In [18]:
prompt_sequence = ["_"] * len(template_gfp.sequence)
prompt_sequence[59] = "T"
prompt_sequence[62] = "T"
prompt_sequence[63] = "Y"
prompt_sequence[64] = "G"
prompt_sequence[93] = "R"
prompt_sequence[219] = "E"
prompt_sequence = "".join(prompt_sequence)

prompt_sequence

'___________________________________________________________T__TYG____________________________R_____________________________________________________________________________________________________________________________E_______'

In [19]:
prompt = model.encode(
    ESMProtein(sequence=prompt_sequence)
)

In [23]:
  # "# We construct an empty structure track like |<bos> <mask> ... <mask> <eos>|...\n",
  #       "prompt.structure = torch.full_like(prompt.sequence, 4096)\n",
  #       "prompt.structure[0] = 4098\n",
  #       "prompt.structure[-1] = 4097\n",
  #       "# ... and then we fill in structure tokens at key residues near the alpha helix\n",
  #       "# kink and at the stabilizing R and E positions on the beta barrel.\n",
  #       "prompt.structure[55:70] = template_gfp_tokens.structure[56:71]\n",
  #       "prompt.structure[93] = template_gfp_tokens.structure[93]\n",
  #       "prompt.structure[219] = template_gfp_tokens.structure[219]\n",
  #       "\n",
  #       "print(\"\".join([\"✔\" if st < 4096 else \"_\" for st in prompt.structure]))"

# construct empty structure track like |<bos> <mask> ... <mask> <eos>|...
prompt.structure = torch.full_like(prompt.sequence, 4096)
prompt.structure[0] = 4098
prompt.structure[-1] = 4097

# fill in structure tokens at key residues near the alpha helix
prompt.structure[55:70] = template_gfp_tokens.structure[56:71]
prompt.structure[93] = template_gfp_tokens.structure[93]
prompt.structure[219] = template_gfp_tokens.structure[219]

print("".join(['x' if st < 4096 else '_' for st in prompt.structure]))

_______________________________________________________xxxxxxxxxxxxxxx_______________________x_____________________________________________________________________________________________________________________________x_________


In [24]:
prompt

ESMProteinTensor(sequence=tensor([ 0, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 11, 32, 32, 11, 19,  6, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 10, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32

> The output shows the original 1qy3 sequence and the our prompt sequence track amino acid identities and the positions that have a token on the structure track. ESM3 will then be tasked with filling in the structure and sequence at the remaining masked (underscore) positions.

Chat GPT 


The notation used to describe the mutation "A93R" is called the single-letter amino acid substitution notation. This format is a common and concise way to indicate changes in protein sequences and is widely used in bioinformatics, molecular biology, and genetics.

Here's a breakdown of the components in this notation:

First letter (A): Represents the original amino acid at the specific position, using the single-letter amino acid code.
Number (93): Indicates the position in the protein sequence where the substitution occurs.
Second letter (R): Represents the new amino acid that replaces the original, also using the single-letter amino acid code.
This notation provides a quick reference to specific mutations, making it easier to discuss and analyze changes in protein sequences across research and clinical settings.

In [25]:
num_tokens_to_decode = (prompt.structure == 4096).sum().item()

structure_generation = model.generate(
    prompt,
    GenerationConfig(
        # Generate a structure.
        track="structure",
        # Sample one token per forward pass of the model.
        num_steps=num_tokens_to_decode,
        # Sampling temperature trades perplexity with diversity.
        temperature=1.0,
    )
)

100%|██████████| 210/210 [09:36<00:00,  2.75s/it]


In [26]:
structure_generation_protein = model.decode(structure_generation)

In [27]:
view = py3Dmol.view(width=1000, height=500)
view.addModel(structure_generation_protein.to_protein_chain().infer_oxygen().to_pdb_string(), "pdb")
view.setStyle({"cartoon": {"color": "lightgreen"}})
view.zoomTo()
view.show()
     

3Dmol.js failed to load for some reason. Please check your browser console for error messages.